# Analyze and Visualize Results

Developed to begin probing at experiment results by looking at a single run.

In [1]:
import logging
import json
from pathlib import Path
from typing import List, Tuple, Union, Dict, Optional
import sys
from multiprocessing import Process, Pool
import psutil
import base64
import asyncio
import cv2
import imageio as iio
import numpy as np
from IPython.core.display import HTML
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, fftn, fftshift

from afdme import REPO_PATH, log
from afdme.vis import (calc_tfpnr, calc_tdr, calc_frames_removed, label_to_per_frame_list)
from afdme.metrics import (
    calc_box_area,
    boxes_to_binary,
    target_detection_rate,
    tfpnr,
)

[DEBUG] 07/02/2023 09:36:14AM: MainProcess: __init__.py - Loaded logging config file: /Users/tsnowak/Projects/afd-me/src/afdme/logging.yaml


### Load experiment output

In [5]:
base_path = f"{REPO_PATH}/scripts/experiments/dft/outputs/"
#exp_path = "2022-10-27/10-57-06/0"  # no tracklet - best
#exp_path = "2022-10-27/13-06-28/0"  # tracklet low fpr - best FPR
#exp_path = "2022-10-27/13-35-41/0"   # tracklet low fnr
#exp_path = "2022-10-28/13-48-14/0"  # tracklet w/ params as no tracklet
exp_path = "2022-11-01/12-26-08/0" # optimized
exp_path = Path(base_path + exp_path)

all_results = []
for f_name in Path(exp_path).glob("**/*.json"):
    with open(f_name, 'r') as f:
        params = json.load(f)
        all_results.append(params)
        

### Calculate metrics on experiment data

In [6]:
# calculate per video scores


all_neg_dets= 0
all_neg_lab= 0
all_frames =0
log.info("Per frame:")
log.info("id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed")
for result in all_results:
    binary_label, binary_pred, tfpnr_dict = calc_tfpnr(result['label'], result['prediction'], show=False, save=False)
    unique_targs, det_targs, tdr = calc_tdr(result['label'], result['prediction'])
    _, n_pos_dets, n_neg_dets = calc_frames_removed(result['prediction'])
    _, n_pos_lab, n_neg_lab = calc_frames_removed(label_to_per_frame_list(result['label']))
    perc_frames_removed = 100*(n_neg_dets / len(binary_pred))
    perc_neg_frame_removed = 100*(n_neg_dets / n_neg_lab)
    all_neg_dets += n_neg_dets
    all_neg_lab += n_neg_lab
    all_frames += len(binary_pred)
    log.info(f"\n{result['label']['video_id']}: {tfpnr_dict['fpr']:.2f}, {tfpnr_dict['fnr']:.2f}, {tdr:.2f}, {perc_frames_removed:.2f}, {perc_neg_frame_removed:.2f}")
perc_all_frames = 100*(all_neg_dets/all_frames)
perc_tn_frames = 100*(all_neg_lab/all_frames)
log.info(f"\n% of all frames removed: {perc_all_frames:.2f}")
log.info(f"\n% of all frames that are empty: {perc_tn_frames:.2f}")

Per frame:
id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed

32: 0.21, 0.20, 1.00, 71.79, 82.35

34: 0.50, 0.04, 1.00, 38.33, 51.49

33: 0.37, 0.00, 1.00, 54.82, 62.81

20: 0.05, 0.43, 1.00, 80.77, 110.53

18: 0.90, 0.03, 1.00, 9.00, 11.05

27: 0.94, 0.02, 1.00, 4.88, 6.06

29: 0.73, 0.00, 1.00, 26.29, 27.32

16: 1.00, 0.00, 1.00, 0.00, 0.00

17: 0.96, 0.00, 1.00, 3.03, 3.70

10: 0.74, 0.07, 1.00, 14.96, 35.19

19: 0.03, 0.70, 1.00, 94.05, 106.76

26: 0.18, 0.14, 1.00, 73.71, 83.33

21: 0.08, 0.29, 1.00, 84.48, 96.08

31: 0.17, 0.13, 1.00, 75.27, 84.19

30: 0.59, 0.08, 1.00, 35.57, 42.06

23: 0.17, 0.46, 0.94, 77.29, 90.88

15: 0.99, 0.00, 1.00, 1.11, 1.11

13: 0.83, 0.00, 1.00, 13.75, 16.67

14: 1.00, 0.00, 1.00, 0.00, 0.00

22: 0.08, 0.27, 0.85, 78.62, 99.21

25: 0.02, 0.08, 1.00, 77.27, 100.00

% of all frames removed: 40.53

% of all frames that are empty: 83.94
